In [1]:
! pip install pandas tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
!nvidia-smi

Tue Nov 25 11:11:23 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   58C    P0              72W / 300W |   5981MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/workspace/Approach1


In [3]:
# !pip install datasets==3.6.0

In [6]:
from transformers import (
    AutoTokenizer,
    MPNetForTokenClassification,
    Trainer,
    TrainingArguments
)
import torch

# -------------------------------
# 1. Device Setup
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------------
# 2. Labels
# -------------------------------
num_labels = len(label_list)
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}





num_labels = len(label_list)
label_to_id = {l: i for i, l in enumerate(label_list)}
id_to_label = {i: l for l, i in label_to_id.items()}

tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")

model = MPNetForTokenClassification.from_pretrained(
    "microsoft/mpnet-base",
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
).to(device)

Using device: cuda


Some weights of MPNetForTokenClassification were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import re
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import BertTokenizerFast
import os
from tqdm import tqdm

# 1. define punctuation map
punctuation_map = {
    ',': 'COMMA',
    '.': 'PERIOD',
    '?': 'QUESTION',
    '!': 'EXCLAMATION',
    ';': 'SEMICOLON',
    ':': 'COLON',
    '-': 'HYPHEN',
    '–': 'EN_DASH',
    '—': 'EM_DASH',
    '(': 'LEFT_PAREN',
    ')': 'RIGHT_PAREN',
    '[': 'LEFT_BRACKET',
    ']': 'RIGHT_BRACKET',
    '{': 'LEFT_BRACE',
    '}': 'RIGHT_BRACE',
    '"': 'DOUBLE_QUOTE',
    "'": 'SINGLE_QUOTE',
    '…': 'ELLIPSIS',
    '/': 'SLASH',
    '\\': 'BACKSLASH',
    '@': 'AT_SYMBOL',
    '#': 'HASH',
    '$': 'DOLLAR',
    '%': 'PERCENT',
    '&': 'AMPERSAND',
    '*': 'ASTERISK',
    '+': 'PLUS',
    '=': 'EQUALS',
    '<': 'LESS_THAN',
    '>': 'GREATER_THAN',
    '|': 'PIPE',
    '^': 'CARET',
    '`': 'BACKTICK',
    '~': 'TILDE'
}

# Automatically create label_list from punctuation_map
label_list = ["O"] + list(punctuation_map.values())
label_to_id = {l: i for i, l in enumerate(label_list)}

print("Label list:", label_list)

Label list: ['O', 'COMMA', 'PERIOD', 'QUESTION', 'EXCLAMATION', 'SEMICOLON', 'COLON', 'HYPHEN', 'EN_DASH', 'EM_DASH', 'LEFT_PAREN', 'RIGHT_PAREN', 'LEFT_BRACKET', 'RIGHT_BRACKET', 'LEFT_BRACE', 'RIGHT_BRACE', 'DOUBLE_QUOTE', 'SINGLE_QUOTE', 'ELLIPSIS', 'SLASH', 'BACKSLASH', 'AT_SYMBOL', 'HASH', 'DOLLAR', 'PERCENT', 'AMPERSAND', 'ASTERISK', 'PLUS', 'EQUALS', 'LESS_THAN', 'GREATER_THAN', 'PIPE', 'CARET', 'BACKTICK', 'TILDE']


In [8]:
# -------------------------------
# 2. Token-label creation function
# -------------------------------
def create_token_labels(sentence):
    tokens = []
    labels = []
    parts = re.findall(r"\w+|[^\w\s]", sentence)
    for i, part in enumerate(parts):
        if re.match(r"\w+", part):  # token
            tokens.append(part)
            if i+1 < len(parts) and parts[i+1] in punctuation_map:
                labels.append(punctuation_map[parts[i+1]])
            else:
                labels.append("O")
    return tokens, labels

# -------------------------------
# 3. Load CSV and create token-label dataset
# -------------------------------
def load_and_process(csv_file):
    df = pd.read_csv(csv_file)
    all_tokens = []
    all_labels = []

    for sent in df['text']:
        tokens, labels = create_token_labels(str(sent))
        all_tokens.append(tokens)
        all_labels.append([label_to_id[l] for l in labels])

    return Dataset.from_dict({"tokens": all_tokens, "labels": all_labels})

print("📄 Processing train/val/test datasets...")
train_dataset = load_and_process("iwslt2017_en_train.csv")
val_dataset   = load_and_process("iwslt2017_en_valid.csv")
test_dataset  = load_and_process("iwslt2017_en_test.csv")

def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(batch["tokens"], is_split_into_words=True, truncation=True, padding="max_length", max_length=128)
    new_labels = []
    for i, label in enumerate(batch["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)
            else:
                aligned_labels.append(label[word_id])
        new_labels.append(aligned_labels)
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
val_dataset   = val_dataset.map(tokenize_and_align_labels, batched=True)
test_dataset  = test_dataset.map(tokenize_and_align_labels, batched=True)

# Remove original columns
train_dataset = train_dataset.remove_columns(["tokens"])
val_dataset   = val_dataset.remove_columns(["tokens"])
test_dataset  = test_dataset.remove_columns(["tokens"])

# -------------------------------
# 5. Set format for PyTorch
# -------------------------------
train_dataset.set_format(type="torch")
val_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

# # -------------------------------
# # 6. Save processed datasets
# # -------------------------------
# os.makedirs("processed_datasets", exist_ok=True)
# train_dataset.save_to_disk("processed_datasets/train")
# val_dataset.save_to_disk("processed_datasets/val")
# test_dataset.save_to_disk("processed_datasets/test")

# print("✅ Processed datasets saved under 'processed_datasets/'")
# print("You can load them later using `Dataset.load_from_disk()`")

📄 Processing train/val/test datasets...


Map: 100%|█████████████████████| 8079/8079 [00:00<00:00, 8601.70 examples/s]


In [15]:
# from datasets import load_from_disk

# train_dataset = load_from_disk("processed_datasets/train")
# val_dataset   = load_from_disk("processed_datasets/val")
# test_dataset  = load_from_disk("processed_datasets/test")

In [9]:
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

206112
888
8079


In [10]:
import numpy as np
import torch
from datasets import load_from_disk



from seqeval.metrics import f1_score, precision_score, recall_score




# -------------------------------
# 4. Metrics for Token Classification
# -------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    true_labels = []
    pred_labels = []

    for label_row, pred_row in zip(labels, predictions):
        true_row = []
        pred_row_labels = []

        for l, p in zip(label_row, pred_row):
            if l != -100:  # ignore padding
                true_row.append(id_to_label[l])
                pred_row_labels.append(id_to_label[p])

        true_labels.append(true_row)
        pred_labels.append(pred_row_labels)

    return {
        "f1": f1_score(true_labels, pred_labels),
        "precision": precision_score(true_labels, pred_labels),
        "recall": recall_score(true_labels, pred_labels),
    }

# -------------------------------
# 5. Training Arguments
# -------------------------------
training_args = TrainingArguments(
    output_dir="./mpnet_token_cls_model",
    eval_strategy="steps",
    eval_steps=1000,
    logging_steps=200,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
)

# -------------------------------
# 6. Trainer
# -------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# -------------------------------
# 7. Train
# -------------------------------
trainer.train()

# -------------------------------
# 8. Save final model
# -------------------------------
trainer.save_model("./mpnet_token_cls_model")
tokenizer.save_pretrained("./mpnet_token_cls_model")

print("Training completed and model saved.")

/tmp/ipykernel_27037/3412898071.py:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,F1,Precision,Recall
1000,0.174000,0.176599,0.793039,0.813748,0.773358
2000,0.140800,0.146941,0.802960,0.814259,0.791971
3000,0.132600,0.131267,0.818709,0.839923,0.798540
4000,0.128300,0.130777,0.816901,0.818849,0.814964
5000,0.125900,0.126962,0.819496,0.832455,0.806934
6000,0.120000,0.122421,0.816198,0.827211,0.805474
7000,0.107200,0.122114,0.821494,0.820000,0.822993
8000,0.106800,0.121598,0.820794,0.823357,0.818248
9000,0.102200,0.125575,0.823444,0.818838,0.828102
10000,0.103400,0.121669,0.826652,0.829232,0.824088


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: COMMA seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PERIOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SINGLE_QUOTE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HYPHEN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: COLON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_l

Training completed and model saved.


In [11]:
trainer.push_to_hub("thenlpresearcher/microsoft_mpnet_punct_model")

Processing Files (0 / 0): |                    |  0.00B /  0.00B            
Processing Files (3 / 4):   0%|                |  640kB /  436MB,  345kB/s  
Processing Files (3 / 4):   0%|                | 1.19MB /  436MB,  614kB/s  
Processing Files (3 / 4):   0%|                | 1.75MB /  436MB,  829kB/s  
Processing Files (3 / 4):   1%|                | 2.85MB /  436MB, 1.26MB/s  
Processing Files (3 / 4):   1%|                | 3.40MB /  436MB, 1.38MB/s  
Processing Files (3 / 4):   1%|▏               | 5.06MB /  436MB, 1.91MB/s  
Processing Files (3 / 4):   2%|▏               | 6.72MB /  436MB, 2.37MB/s  
Processing Files (3 / 4):   3%|▍               | 12.8MB /  436MB, 4.24MB/s  
Processing Files (3 / 4):   5%|▊               | 21.1MB /  436MB, 6.56MB/s  
Processing Files (3 / 4):   8%|█▎              | 34.4MB /  436MB, 10.1MB/s  
Processing Files (3 / 4):  10%|█▌              | 42.6MB /  436MB, 11.8MB/s  
Processing Files (3 / 4):  11%|█▊              | 49.8MB /  436MB, 13.1MB/s  

CommitInfo(commit_url='https://huggingface.co/thenlpresearcher/mpnet_token_cls_model/commit/f74c1d02b55098fc00eecc66efb6ac38eb29deec', commit_message='thenlpresearcher/microsoft_mpnet_punct_model', commit_description='', oid='f74c1d02b55098fc00eecc66efb6ac38eb29deec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/thenlpresearcher/mpnet_token_cls_model', endpoint='https://huggingface.co', repo_type='model', repo_id='thenlpresearcher/mpnet_token_cls_model'), pr_revision=None, pr_num=None)

In [12]:
# -------------------------------
# Evaluate on test set
# -------------------------------
test_metrics = trainer.evaluate(eval_dataset=test_dataset)
print("✅ Test set evaluation:")
print(f"F1 Score: {test_metrics['eval_f1']:.4f}")
print(f"Precision: {test_metrics['eval_precision']:.4f}")
print(f"Recall: {test_metrics['eval_recall']:.4f}")

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: COMMA seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PERIOD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SINGLE_QUOTE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: COLON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: HYPHEN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_l

✅ Test set evaluation:
F1 Score: 0.8800
Precision: 0.8838
Recall: 0.8763


In [13]:
test_metrics

{'eval_loss': 0.09630389511585236,
 'eval_f1': 0.8800178412132025,
 'eval_precision': 0.8837625979843226,
 'eval_recall': 0.8763046857650455,
 'eval_runtime': 6.6652,
 'eval_samples_per_second': 1212.113,
 'eval_steps_per_second': 37.958,
 'epoch': 4.0}

In [14]:
import numpy as np
from seqeval.metrics import classification_report

# Get predictions
predictions, labels, _ = trainer.predict(test_dataset)
pred_ids = np.argmax(predictions, axis=-1)

true_labels_list = []
pred_labels_list = []

for label_row, pred_row in zip(labels, pred_ids):
    true_row = []
    pred_row_labels = []
    for l, p in zip(label_row, pred_row):
        if l != -100:  # ignore padding
            true_row.append(label_list[l])
            pred_row_labels.append(label_list[p])
    true_labels_list.append(true_row)
    pred_labels_list.append(pred_row_labels)

# Generate detailed report
print("\nDetailed classification report (per label):")
print(classification_report(true_labels_list, pred_labels_list, digits=4))


Detailed classification report (per label):


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         ASH     0.0000    0.0000    0.0000         2
 EFT_BRACKET     0.0000    0.0000    0.0000        15
    EMICOLON     0.6275    0.2424    0.3497       132
       ERIOD     0.9652    0.9736    0.9694      7830
IGHT_BRACKET     0.0000    0.0000    0.0000        26
 INGLE_QUOTE     0.9957    0.9978    0.9968      3720
        LASH     1.0000    0.3333    0.5000         9
         LUS     0.0000    0.0000    0.0000         2
    MPERSAND     1.0000    1.0000    1.0000         8
      M_DASH     0.0000    0.0000    0.0000        27
       OLLAR     0.8333    0.6250    0.7143         8
        OLON     0.6255    0.6255    0.6255       259
        OMMA     0.7897    0.8113    0.8004      8460
 OUBLE_QUOTE     0.6147    0.5425    0.5763       247
       QUALS     0.0000    0.0000    0.0000         2
     UESTION     0.9111    0.9147    0.9129       739
  XCLAMATION     0.7692    0.1724    0.2817        58
       YPHEN     0.7086    

In [15]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer

# -------------------------------
# 1. Device setup
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------------
# 2. Load tokenizer and model
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained("thenlpresearcher/mpnet_token_cls_model")
model = AutoModelForTokenClassification.from_pretrained("thenlpresearcher/mpnet_token_cls_model")
model.to(device)

# -------------------------------
# 3. Prepare Trainer (only for prediction)
# -------------------------------
trainer = Trainer(
    model=model,
    tokenizer=tokenizer
)

# -------------------------------
# 4. Make predictions on test dataset
# -------------------------------
predictions, labels, _ = trainer.predict(test_dataset)  # test_dataset must be already prepared
pred_ids = np.argmax(predictions, axis=-1)

Using device: cuda


/tmp/ipykernel_27037/1791168934.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
# -------------------------------
# 5. Map predictions and labels back to strings
# -------------------------------
true_labels_list = []
pred_labels_list = []

for label_row, pred_row in zip(labels, pred_ids):
    true_row = []
    pred_row_labels = []
    for l, p in zip(label_row, pred_row):
        if l != -100:  # ignore padding
            true_row.append(label_list[l])
            pred_row_labels.append(label_list[p])
    true_labels_list.append(true_row)
    pred_labels_list.append(pred_row_labels)

In [17]:
from sklearn.metrics import classification_report

# Flatten lists for sklearn
y_true_flat = [label for seq in true_labels_list for label in seq]
y_pred_flat = [label for seq in pred_labels_list for label in seq]

print("\nDetailed classification report (per label) using sklearn:")
print(classification_report(y_true_flat, y_pred_flat, digits=4))


Detailed classification report (per label) using sklearn:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


               precision    recall  f1-score   support

    AMPERSAND     1.0000    1.0000    1.0000         8
        COLON     0.6407    0.6473    0.6440       292
        COMMA     0.8124    0.8376    0.8248      9916
       DOLLAR     0.8333    0.6250    0.7143         8
 DOUBLE_QUOTE     0.6606    0.5783    0.6167       313
      EM_DASH     0.0000    0.0000    0.0000        27
       EQUALS     0.0000    0.0000    0.0000         2
  EXCLAMATION     0.7692    0.1724    0.2817        58
         HASH     0.0000    0.0000    0.0000         2
       HYPHEN     0.7160    0.5237    0.6050      1117
 LEFT_BRACKET     0.0000    0.0000    0.0000        15
            O     0.9860    0.9875    0.9867    117294
       PERIOD     0.9667    0.9737    0.9702      8729
         PLUS     0.0000    0.0000    0.0000         2
     QUESTION     0.9193    0.9170    0.9181       795
RIGHT_BRACKET     0.0000    0.0000    0.0000        34
    SEMICOLON     0.6154    0.2424    0.3478       132
 SINGLE_Q

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
punctuation_reverse_map = {v: k for k, v in punctuation_map.items()}
punctuation_reverse_map["O"] = ""   # no punctuation

In [20]:
import os
print(os.getcwd())

/workspace/Approach1


In [9]:
import torch

def restore_punctuation(text, tokenizer, model, label_list, device=None):
    """
    text: raw unpunctuated string
    tokenizer: HF tokenizer
    model: punctuation restoration model (token classification)
    label_list: list mapping label IDs -> label strings
    """

    # Select device automatically if not provided
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Tokenize on CPU (normal), push tensors to GPU
    encoded = tokenizer(
        text.split(),            # token classification expects token-wise input
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    encoded = {k: v.to(device) for k, v in encoded.items()}

    # Run model on GPU
    with torch.no_grad():
        logits = model(**encoded).logits
        predictions = torch.argmax(logits, dim=-1)[0].cpu().tolist()  
        # convert back to CPU to handle Python label mapping

    tokens = text.split()
    punctuated_tokens = []

    for token, pred_id in zip(tokens, predictions):
#         print(pred_id)
        label = label_list[pred_id]
        print(f"{label}, {token}, {pred_id}")
        # No-punctuation label → just append token
        if label == "O":
            punctuated_tokens.append(token)
        else:
            # label looks like: COMMA, PERIOD, QUESTION
            punct_char = punctuation_reverse_map.get(label, "")
            punctuated_tokens.append(token + punct_char)

    return " ".join(punctuated_tokens)


restore_punctuation("I am good what about you", tokenizer, model, label_list, device)

NameError: name 'model' is not defined

O, I, 0
O, am, 0
O, good, 0
O, what, 0
O, about, 0
COMMA, you, 1


'I am good what about you,'

{'labels': tensor([-100,    0,    0,    0,    0,    0,    1,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    2, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]),
 'input_ids': tensor([ 101, 2195, 2086, 3283, 2182, 2012, 6945, 2848, 8066, 2386, 3107, 1037,
         2640, 4119, 2170, 